# **Google Play Store Data Cleaning & Preparation**

This notebook focuses on cleaning and preparing a dataset of apps from the Google Play Store. The goal is to handle missing values, correct data types, and clean any inconsistencies to make the dataset suitable for analysis.

In [1]:
import pandas as pd 
import numpy as np
import re

## 📥 Load Dataset
Load the Google Play Store dataset into a Pandas DataFrame.

In [3]:
df= pd.read_csv(r"C:\Users\HP\Documents\Google-Playstore.csv",encoding= 'utf-8')

## Step 2: Explore the Dataset
- Examine the dataset for missing values, duplicates, and incorrect data types.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2312944 entries, 0 to 2312943
Data columns (total 24 columns):
 #   Column             Dtype  
---  ------             -----  
 0   App Name           object 
 1   App Id             object 
 2   Category           object 
 3   Rating             float64
 4   Rating Count       float64
 5   Installs           object 
 6   Minimum Installs   float64
 7   Maximum Installs   int64  
 8   Free               bool   
 9   Price              float64
 10  Currency           object 
 11  Size               object 
 12  Minimum Android    object 
 13  Developer Id       object 
 14  Developer Website  object 
 15  Developer Email    object 
 16  Released           object 
 17  Last Updated       object 
 18  Content Rating     object 
 19  Privacy Policy     object 
 20  Ad Supported       bool   
 21  In App Purchases   bool   
 22  Editors Choice     bool   
 23  Scraped Time       object 
dtypes: bool(4), float64(4), int64(1), object(15)
memor

In [7]:
df.head()

,App Name,App Id,Category,Rating,Rating Count,Installs,Minimum Installs,Maximum Installs,Free,Price,...,Developer Website,Developer Email,Released,Last Updated,Content Rating,Privacy Policy,Ad Supported,In App Purchases,Editors Choice,Scraped Time
0,Gakondo,com.ishakwe.gakondo,Adventure,0.0,0.0,10+,10.0,15,True,0.0,...,https://beniyizibyose.tk/#/,jean21101999@gmail.com,"Feb 26, 2020","Feb 26, 2020",Everyone,https://beniyizibyose.tk/projects/,False,False,False,2021-06-15 20:19:35
1,Ampere Battery Info,com.webserveis.batteryinfo,Tools,4.4,64.0,"5,000+",5000.0,7662,True,0.0,...,https://webserveis.netlify.app/,webserveis@gmail.com,"May 21, 2020","May 06, 2021",Everyone,https://dev4phones.wordpress.com/licencia-de-uso/,True,False,False,2021-06-15 20:19:35
2,Vibook,com.doantiepvien.crm,Productivity,0.0,0.0,50+,50.0,58,True,0.0,...,NaN,vnacrewit@gmail.com,"Aug 9, 2019","Aug 19, 2019",Everyone,https://www.vietnamairlines.com/vn/en/terms-an...,False,False,False,2021-06-15 20:19:35
3,Smart City Trichy Public Service Vehicles 17UC...,cst.stJoseph.ug17ucs548,Communication,5.0,5.0,10+,10.0,19,True,0.0,...,http://www.climatesmarttech.com/,climatesmarttech2@gmail.com,"Sep 10, 2018","Oct 13, 2018",Everyone,NaN,True,False,False,2021-06-15 20:19:35
4,GROW.me,com.horodyski.grower,Tools,0.0,0.0,100+,100.0,478,True,0.0,...,http://www.horodyski.com.pl,rmilekhorodyski@gmail.com,"Feb 21, 2020","Nov 12, 2018",Everyone,http://www.horodyski.com.pl,False,False,False,2021-06-15 20:19:35


## Step 3: Data Cleaning

Clean and transform the dataset:

- Remove or fill missing values  
- Drop duplicates  
- Convert columns to appropriate data types  
- Clean text and numeric columns (e.g., remove symbols, ommas)


In [9]:
df.shape

(2312944, 24)

🔎 Initial Exploration
Explore the dataset structure and identify missing or duplicate values.

In [11]:
df.isnull().sum()

App Name                  5
App Id                    0
Category                  0
Rating                22883
Rating Count          22883
Installs                107
Minimum Installs        107
Maximum Installs          0
Free                      0
Price                     0
Currency                135
Size                    196
Minimum Android        6530
Developer Id             33
Developer Website    760835
Developer Email          31
Released              71053
Last Updated              0
Content Rating            0
Privacy Policy       420953
Ad Supported              0
In App Purchases          0
Editors Choice            0
Scraped Time              0
dtype: int64

In [13]:
#Checks for the columns name in the dataframe 
df.columns

Index(['App Name', 'App Id', 'Category', 'Rating', 'Rating Count', 'Installs',
       'Minimum Installs', 'Maximum Installs', 'Free', 'Price', 'Currency',
       'Size', 'Minimum Android', 'Developer Id', 'Developer Website',
       'Developer Email', 'Released', 'Last Updated', 'Content Rating',
       'Privacy Policy', 'Ad Supported', 'In App Purchases', 'Editors Choice',
       'Scraped Time'],
      dtype='object')

In [15]:
#Removes null values from the App Name column 
df = df.dropna(subset = ['App Name'])

In [17]:
# Removing null values from the install column
df = df.dropna(subset = ['Installs'])

In [19]:
# Remove the + sign in the install values 
df['Installs'] = df['Installs']. apply(lambda x: x.replace('+', '') if '+' in str(x) else x)

In [21]:
#Remooves ',' in the install values
df['Installs'] = df['Installs']. apply(lambda x: x.replace(',', '') if ',' in str(x) else x)

In [23]:
#Convert the dtype of installs from objects to int 
df['Installs'] = df['Installs']. astype('int64')

In [25]:
def convert_size_to_numeric(size):
    """
    Convert size values from string format (e.g., '15M', '1.2G', '500k') 
    to numeric format in bytes.

    Parameters:
    size (str or numeric): The size value from the dataset. Can be a string 
    with units ('k', 'M', 'G') or already numeric.

    Returns:
    float: The converted size value in bytes. Returns NaN if the value is 
    'Varies with device'.
    """
    
    if isinstance(size, str):  # Check if the input is a string
        size = size.replace(',', '')  # Remove commas for consistency
        
        # Convert based on size unit
        if 'k' in size:
            return float(size.replace('k', '')) * 1024  # Convert KB to bytes
        elif 'M' in size:
            return float(size.replace('M', '')) * 1024 * 1024  # Convert MB to bytes
        elif 'G' in size:
            return float(size.replace('G', '')) * 1024 ** 3  # Convert GB to bytes
        elif 'Varies with device' in size:
            return np.nan  # Assign NaN for inconsistent values
        else:
            return float(size)  # If no unit, return as float
    
    return size  # If already numeric, return as is


In [27]:
df.rename(columns={'Size': 'Size_in_bytes'}, inplace=True)

In [29]:
# Apply the function to convert size values to numeric format
df['Size_in_bytes'] = df['Size_in_bytes'].apply(convert_size_to_numeric)

In [35]:
df['Size_MB'] = df['Size_in_bytes'] / (1024 * 1024)  # Convert bytes to MB
df['Size_MB'] = df['Size_MB'].round(2)  # Round to 2 decimal places

In [37]:
df['Released'] = pd.to_datetime(df['Released'], errors='coerce')

In [39]:
df['Last Updated'] = pd.to_datetime(df['Last Updated'], errors='coerce')

In [41]:
#Filling in missing values 
# Use median where appropriate
df['Rating'] = df['Rating'].fillna(df['Rating'].median())  # Approximate missing ratings
df['Rating Count'] = df['Rating Count'].fillna(df['Rating Count'].median())  # No rating count = 0
df['Size_in_bytes'] = df['Size_in_bytes'].fillna(df['Size_in_bytes'].median())  # Approximate missing sizes
df['Released'] = df['Released'].fillna(df['Released'].median()) 
df['Currency'] = df['Currency'].fillna('Unknown')

In [43]:
df.drop(columns=['Scraped Time'], inplace=True, errors='ignore')

In [45]:
df.drop(columns=[ 'Developer Id', 'Developer Website',
       'Developer Email'], inplace=True, errors='ignore')

In [47]:
df.drop(columns=['Privacy Policy'], inplace=True, errors='ignore')

In [49]:
df.rename( columns = {'Free': 'Type'}, inplace = True) 

In [51]:
df['Type'] = df['Type'].replace({True: 'Free', False: 'Paid'})

In [53]:
df.isnull().sum()

App Name                0
App Id                  0
Category                0
Rating                  0
Rating Count            0
Installs                0
Minimum Installs        0
Maximum Installs        0
Type                    0
Price                   0
Currency                0
Size_in_bytes           0
Minimum Android      6530
Released                0
Last Updated            0
Content Rating          0
Ad Supported            0
In App Purchases        0
Editors Choice          0
Size_MB             74905
dtype: int64

In [55]:
df['Minimum Android'].unique()

array(['7.1 and up', '5.0 and up', '4.0.3 and up', '4.1 and up',
       '6.0 and up', '4.4 and up', '4.0 and up', '4.2 and up',
       '2.1 and up', '7.0 and up', '2.2 and up', '2.3 and up',
       '4.3 and up', '5.1 and up', nan, '1.6 and up', '3.0 and up',
       'Varies with device', '8.0 and up', '2.3.3 and up', '4.4W and up',
       '3.2 and up', '1.5 and up', '4.0.3 - 7.1.1', '3.1 and up',
       '2.0 and up', '1.0 and up', '1.1 and up', '2.0.1 and up',
       '4.4 - 6.0', '2.1 - 4.4', '4.0 - 5.0', '2.0 - 2.3.4', '2.2 - 4.4',
       '4.1 - 7.0', '4.1 - 6.0', '4.0 - 4.4W', '4.0 - 4.4', '4.0.3 - 7.0',
       '2.3 - 5.0', '4.0 - 7.1.1', '4.1 - 7.1.1', '4.0 - 7.0',
       '4.1 - 8.0', '4.0 - 6.0', '2.3 - 7.0', '4.0 - 8.0', '2.3 - 6.0',
       '2.1 - 2.3.4', '3.0 - 4.1.1', '2.2 - 3.0', '2.0 - 8.0',
       '4.0.3 - 8.0', '3.0 - 4.4W', '4.4 - 7.1.1', '4.4 - 8.0',
       '4.0.3 - 6.0', '1.6 - 4.0.4', '8.0', '2.3 - 5.1', '2.1 - 7.1.1',
       '2.3 - 4.4W', '4.4 - 7.0', '6.0 - 7.1.1', '2.3

In [57]:
# Function to clean "Minimum Android"
def clean_minimum_android(value):
    if pd.isna(value) or value == "Varies with device":
        return np.nan  # Convert missing and "Varies with device" to NaN
    
    value = str(value).split(" and up")[0]  # Remove " and up"
    
    if " - " in value:  # If range exists, take the lower bound
        value = value.split(" - ")[0]
    
    value = value.replace("W", "").strip()  # Remove "W" and unnecessary spaces
    
    return value if value else np.nan  # Keep as string (object) instead of float

# Apply function to dataset
df['Minimum Android'] = df['Minimum Android'].apply(clean_minimum_android)

In [59]:
# Group by 'Category' and fill missing values with the most common value (mode)
df['Minimum Android'] = df.groupby('Category')['Minimum Android'].transform(lambda x: x.fillna(x.mode()[0]))

In [61]:
df.isnull().sum()

App Name                0
App Id                  0
Category                0
Rating                  0
Rating Count            0
Installs                0
Minimum Installs        0
Maximum Installs        0
Type                    0
Price                   0
Currency                0
Size_in_bytes           0
Minimum Android         0
Released                0
Last Updated            0
Content Rating          0
Ad Supported            0
In App Purchases        0
Editors Choice          0
Size_MB             74905
dtype: int64

In [63]:
df.duplicated().sum()

0

In [65]:
df.rename(columns={
    'App Name': 'App_Name',
    'App Id': 'App_Id',
    'Category': 'Category',
    'Rating': 'Rating',
    'Rating Count': 'Rating_Count',
    'Installs': 'Installs',
    'Minimum Installs': 'Minimum_Installs',
    'Maximum Installs': 'Maximum_Installs',
    'Type': 'Type',
    'Price': 'Price',
    'Currency': 'Currency',
    'Minimum Android': 'Minimum_Android',
    'Released': 'Released',
    'Last Updated': 'Last_Updated',
    'Content Rating': 'Content_Rating',
    'Ad Supported': 'Ad_Supported',
    'In App Purchases': 'In_App_Purchases',
    'Editors Choice': 'Editors_Choice'
}, inplace=True)


In [67]:
df.head()

,App_Name,App_Id,Category,Rating,Rating_Count,Installs,Minimum_Installs,Maximum_Installs,Type,Price,Currency,Size_in_bytes,Minimum_Android,Released,Last_Updated,Content_Rating,Ad_Supported,In_App_Purchases,Editors_Choice,Size_MB
0,Gakondo,com.ishakwe.gakondo,Adventure,0.0,0.0,10,10.0,15,Free,0.0,USD,10485760.0,7.1,2020-02-26,2020-02-26,Everyone,False,False,False,10.0
1,Ampere Battery Info,com.webserveis.batteryinfo,Tools,4.4,64.0,5000,5000.0,7662,Free,0.0,USD,3040870.4,5.0,2020-05-21,2021-05-06,Everyone,True,False,False,2.9
2,Vibook,com.doantiepvien.crm,Productivity,0.0,0.0,50,50.0,58,Free,0.0,USD,3879731.2,4.0.3,2019-08-09,2019-08-19,Everyone,False,False,False,3.7
3,Smart City Trichy Public Service Vehicles 17UC...,cst.stJoseph.ug17ucs548,Communication,5.0,5.0,10,10.0,19,Free,0.0,USD,1887436.8,4.0.3,2018-09-10,2018-10-13,Everyone,True,False,False,1.8
4,GROW.me,com.horodyski.grower,Tools,0.0,0.0,100,100.0,478,Free,0.0,USD,6501171.2,4.1,2020-02-21,2018-11-12,Everyone,False,False,False,6.2


In [69]:
df.shape 

(2312832, 20)

In [75]:
Cleaned_dataset = df.to_csv(r"C:\Users\HP\Documents\Google_playstore_dataset_cleaned.csv", index = False)

## ✅ Summary

- Loaded and explored the dataset.  
- Handled missing values and duplicates.  
- Cleaned and converted data types.  
- Transformed columns for better usabiliy.  

The dataset has been cleaned by handling missing values, fixing data types, and formatting columns. It's now ready for further analysis or moeling.
